In [1]:
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
import numpy as np
from gspread import Client
from gspread_dataframe import set_with_dataframe
import urllib
import sqlalchemy
import pyodbc
import sys
import geopandas as gpd
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config

In [2]:
config = get_config('c:\Projects\config.ini')
driver = config['srv']['driver']
server = config['srv']['server']
data_parks = config['srv']['data_parks']
dwh = config['db']['dwh']
cred_file = config['google']['path_to_file']

# Read in Google Sheets

In [3]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

In [4]:
session = create_assertion_session(cred_file, scope)

In [5]:
gc = Client(None, session)

In [6]:
# gc = gspread.authorize(credentials)

In [6]:
wks = gc.open("Crowds_Combined").sheet1

In [7]:
wks2 = gc.open("DailyTasks_WebMerc_Centroids").sheet1

In [8]:
# headers_wks = gc.open("Report_Headers").sheet1

In [9]:
data = wks.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)
df.head()

,Timestamp,District,NumberOfPeople,CrowdInPlayground,ActionTakenByParksEmployee,LocationInPark,AdditionalComments,Site,Borough
0,5/15/2020 10:12:47,X-06,20,No,Approached the crowd; they ignored the employee,Bench/sitting area,People crowded on benches,X010A | Tremont Park,Bronx
1,5/15/2020 9:08:54,X-06,4,No,Approached the crowd; they ignored the employee,Bench/sitting area,,X042 | Rose Hill Park,Bronx
2,5/14/2020 18:54:58,Q-13,15,No,Did not approach the crowd; the crowd remains,Soccer field,,Q008-02 | Brookville Park-Brookville Park,Queens
3,5/14/2020 18:05:19,M-06,6,Yes,Approached the crowd; they ignored the employee,Handall court,Patrons stated everybody else is doing it,M164 | Asser Levy Playground,Manhattan
4,5/14/2020 17:49:33,Q-13,21,No,Approached the crowd; they ignored the employee,Adult fitness equipment,,Q008-02 | Brookville Park-Brookville Park,Queens


In [11]:
geom = wks2.get_all_values()
headers_geom = geom.pop(0)

df2 = pd.DataFrame(geom, columns=headers_geom)
df2.head()

,PROPERTY_I,DESCRIPTIO,DISTRICT,DESC_LOCAT,ESRI_OID,Latitiude,Longitude
0,Q102,Juniper Valley Park,Q-05,Q102 | Juniper Valley Park,1,40.7202,-73.8804
1,Q024,Kissena Park,Q-07,Q024 | Kissena Park,2,40.7454,-73.8048
2,Q347,Colden Playground,Q-07,Q347 | Colden Playground,3,40.7702,-73.8272
3,Q121,Detective Keith L Williams Park,Q-12,Q121 | Detective Keith L Williams Park,4,40.7029,-73.7844
4,Q375,Willets Point Playground,Q-07,Q375 | Willets Point Playground,5,40.7838,-73.7964


In [17]:
df2.rename(columns = {'Latitiude':'Latitude'},inplace=True)

In [19]:
df2['Latitude'] = df2['Latitude'].astype(float)

In [20]:
df2['Longitude'] = df2['Longitude'].astype(float)

In [23]:
geom_gdf = gpd.GeoDataFrame(df2,
                 geometry=gpd.points_from_xy(df2.Longitude, df2.Latitude),
                 crs={'init': 'epsg:4326'})

In [24]:
geom_gdf.head()

,PROPERTY_I,DESCRIPTIO,DISTRICT,DESC_LOCAT,ESRI_OID,Latitude,Longitude,geometry
0,Q102,Juniper Valley Park,Q-05,Q102 | Juniper Valley Park,1,40.7202,-73.8804,POINT (-73.88039999999999 40.7202)
1,Q024,Kissena Park,Q-07,Q024 | Kissena Park,2,40.7454,-73.8048,POINT (-73.8048 40.7454)
2,Q347,Colden Playground,Q-07,Q347 | Colden Playground,3,40.7702,-73.8272,POINT (-73.8272 40.7702)
3,Q121,Detective Keith L Williams Park,Q-12,Q121 | Detective Keith L Williams Park,4,40.7029,-73.7844,POINT (-73.78440000000001 40.7029)
4,Q375,Willets Point Playground,Q-07,Q375 | Willets Point Playground,5,40.7838,-73.7964,POINT (-73.79640000000001 40.7838)


In [28]:
MO_Crowds = geom_gdf[['DESC_LOCAT','geometry']].merge(df, how = 'right', left_on = 'DESC_LOCAT', right_on = 'Site')

In [29]:
MO_Crowds.head()

,DESC_LOCAT,geometry,Timestamp,District,NumberOfPeople,CrowdInPlayground,ActionTakenByParksEmployee,LocationInPark,AdditionalComments,Site,Borough
0,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),3/27/2020 15:07:25,Q-05,200,Yes,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
1,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),3/26/2020 14:37:04,Q-05,12,No,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
2,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),3/26/2020 14:36:46,Q-05,8,Yes,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
3,Q024 | Kissena Park,POINT (-73.8048 40.7454),5/4/2020 9:16:57,Q-07,80,No,Did not approach the crowd; the crowd remains,Walking path,,Q024 | Kissena Park,Queens
4,Q024 | Kissena Park,POINT (-73.8048 40.7454),4/19/2020 12:21:33,Q-07,5,No,Did not approach the crowd; the crowd remains,Soccer field,Kissena soccer field team play \n,Q024 | Kissena Park,Queens


# change datatype of Timestamp to datetime

In [30]:
MO_Crowds['Timestamp'] = pd.to_datetime(MO_Crowds['Timestamp'], infer_datetime_format=True)

In [31]:
MO_Crowds

,DESC_LOCAT,geometry,Timestamp,District,NumberOfPeople,CrowdInPlayground,ActionTakenByParksEmployee,LocationInPark,AdditionalComments,Site,Borough
0,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),2020-03-27 15:07:25,Q-05,200,Yes,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
1,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),2020-03-26 14:37:04,Q-05,12,No,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
2,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),2020-03-26 14:36:46,Q-05,8,Yes,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
3,Q024 | Kissena Park,POINT (-73.8048 40.7454),2020-05-04 09:16:57,Q-07,80,No,Did not approach the crowd; the crowd remains,Walking path,,Q024 | Kissena Park,Queens
4,Q024 | Kissena Park,POINT (-73.8048 40.7454),2020-04-19 12:21:33,Q-07,5,No,Did not approach the crowd; the crowd remains,Soccer field,Kissena soccer field team play \n,Q024 | Kissena Park,Queens
5,Q024 | Kissena Park,POINT (-73.8048 40.7454),2020-03-26 12:16:08,Q-07,6,No,Did not approach the crowd; the crowd remains,,,Q024 | Kissena Park,Queens
6,Q024 | Kissena Park,POINT (-73.8048 40.7454),2020-03-25 17:05:47,Q-07,6,No,Did not approach the crowd; the crowd remains,,,Q024 | Kissena Park,Queens
7,Q024 | Kissena Park,POINT (-73.8048 40.7454),2020-03-25 16:58:29,Q-07,5,No,Did not approach the crowd; the crowd remains,,,Q024 | Kissena Park,Queens
8,Q080 | Veterans Plaza,POINT (-73.87479999999999 40.7588),2020-04-16 17:34:38,Q-03,10,No,Did not approach the crowd; the crowd remains,Bench/sitting area,"Includes shopping carts, and bags of cans/bott...",Q080 | Veterans Plaza,Queens
9,Q031 | Lawrence Virgilio Playground,POINT (-73.91079999999999 40.7475),2020-03-26 11:42:44,Q-02,15,Yes,Did not approach the crowd; the crowd remains,,,Q031 | Lawrence Virgilio Playground,Queens


In [36]:
con_string = 'Driver={SQL Server};Server=dpr-vpipm001;Database=dwh;Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [37]:
def to_wkt(row):
    return row.wkt

In [40]:
MO_Crowds.geometry

0       POINT (-73.88039999999999 40.7202)
1       POINT (-73.88039999999999 40.7202)
2       POINT (-73.88039999999999 40.7202)
3                 POINT (-73.8048 40.7454)
4                 POINT (-73.8048 40.7454)
5                 POINT (-73.8048 40.7454)
6                 POINT (-73.8048 40.7454)
7                 POINT (-73.8048 40.7454)
8       POINT (-73.87479999999999 40.7588)
9       POINT (-73.91079999999999 40.7475)
10                POINT (-73.8074 40.7705)
11                POINT (-73.8074 40.7705)
12      POINT (-73.90260000000001 40.7396)
13      POINT (-73.90260000000001 40.7396)
14      POINT (-73.90260000000001 40.7396)
15      POINT (-73.90260000000001 40.7396)
16      POINT (-73.90260000000001 40.7396)
17      POINT (-73.90260000000001 40.7396)
18      POINT (-73.90260000000001 40.7396)
19      POINT (-73.85299999999999 40.7021)
20      POINT (-73.85299999999999 40.7021)
21      POINT (-73.85299999999999 40.7021)
22                POINT (-73.8884 40.6877)
23         

In [42]:
len(df)

1311

In [43]:
len(MO_Crowds)

1312

In [49]:
MO_Crowds_to_publish = MO_Crowds[~pd.isnull(MO_Crowds['geometry'])].copy()

In [50]:
MO_Crowds_to_publish['geometry'] = MO_Crowds_to_publish['geometry'].apply(to_wkt)

In [51]:
MO_Crowds_to_publish.to_sql('tbl_MO_crowds', engine, if_exists='replace')

In [55]:
con = pyodbc.connect('Driver={' + driver + '};Server=' + server +
                     ';Database=' + dwh + ';Trusted_Connection=Yes;')

In [56]:
cursor = con.cursor()
cursor.execute("""
    alter table [dwh].[dbo].[tbl_MO_crowds] 
    alter column geometry geometry
    """)
con.commit()
cursor.close()